In [43]:
#Import Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
from datetime import datetime
import requests
from config import weather_api_key

In [31]:
#Creating variables for longitutde and latitude arrays and then combining them together
lats = np.random.uniform(-90.00,90.00,size=1500)
lngs = np.random.uniform(-180.00,180.00, size=1500)

lat_lngs = zip(lats,lngs)
lat_lngs

In [32]:
#Add coordinates to a list
coordinates = list(lat_lngs)

In [33]:
#Create cities list to store city data from the loop
cities=[]

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    
    #if city is unique then we add it to the list
    if city not in cities:
        cities.append(city)
#print length of cities
len(cities)


591

In [45]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [57]:
city_data=[]

print("Beginning Data Retrieval     ")
print("-----------------------------")

#create counters
record_count = 1
set_count = 1



# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

#Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | tasiilaq
Processing Record 2 of Set 1 | kudahuvadhoo
Processing Record 3 of Set 1 | georgetown
Processing Record 4 of Set 1 | bengkulu
Processing Record 5 of Set 1 | ribeira grande
Processing Record 6 of Set 1 | shorewood
Processing Record 7 of Set 1 | hermanus
Processing Record 8 of Set 1 | avarua
Processing Record 9 of Set 1 | winnemucca
Processing Record 10 of Set 1 | bluff
Processing Record 11 of Set 1 | san cristobal
Processing Record 12 of Set 1 | albany
Processing Record 13 of Set 1 | kadoma
Processing Record 14 of Set 1 | saint-jean-de-la-ruelle
Processing Record 15 of Set 1 | mecca
Processing Record 16 of Set 1 | balky
Processing Record 17 of Set 1 | jamestown
Processing Record 18 of Set 1 | adre
Processing Record 19 of Set 1 | sentyabrskiy
City not found. Skipping...
Processing Record 20 of Set 1 | strezhevoy
Processing Record 21 of Set 1 | pangnirtung
Processing Record 22 of Set 1 | pa

In [58]:
len(city_data)

547

In [61]:
#converting array in dataframe
city_data_df = pd.DataFrame(city_data)
new_column_order = ["City","Country","Date","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Tasiilaq,GL,2020-08-27 16:43:18,65.61,-37.64,39.20,80,20,10.29
1,Kudahuvadhoo,MV,2020-08-27 16:43:18,2.67,72.89,83.53,70,100,11.32
2,Georgetown,MY,2020-08-27 16:43:18,5.41,100.34,80.60,88,20,4.70
3,Bengkulu,ID,2020-08-27 16:43:18,-3.80,102.27,76.17,84,2,5.68
4,Ribeira Grande,PT,2020-08-27 16:43:19,38.52,-28.70,77.00,88,75,5.82
5,Shorewood,US,2020-08-27 16:45:47,41.52,-88.20,91.00,62,1,9.17
6,Hermanus,ZA,2020-08-27 16:45:47,-34.42,19.23,59.00,80,47,1.99
7,Avarua,CK,2020-08-27 16:45:47,-21.21,-159.78,71.60,78,86,3.36
8,Winnemucca,US,2020-08-27 16:43:19,40.97,-117.74,75.20,33,1,6.93
9,Bluff,NZ,2020-08-27 16:45:48,-46.60,168.33,42.53,81,88,12.80


In [62]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")